In [20]:
from tensorflow.keras.models import Model
import tensorflow as tf
from model import lstm_unet
import keras.backend as K
from helpers import f1
import warnings
import numpy as np
import matplotlib.pyplot as plt
from data_loader import DataGenerator
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
   tf.config.set_visible_devices(gpus[0], 'GPU')
   tf.config.experimental.set_memory_growth(gpus[0], True)

In [21]:
model = lstm_unet()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 15, 256, 256, 3)]    0         []                            
                                                                                                  
 conv_lstm2d_18 (ConvLSTM2D  (None, 15, 256, 256, 16)     11008     ['input_3[0][0]']             
 )                                                                                                
                                                                                                  
 batch_normalization_18 (Ba  (None, 15, 256, 256, 16)     64        ['conv_lstm2d_18[0][0]']      
 tchNormalization)                                                                                
                                                                                            

In [22]:
train_gen = DataGenerator(
    image_dir="data_v3_processed/train/images/images",
    mask_dir="data_v3_processed/train/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
    infinite=True,
)

In [23]:
val_gen = DataGenerator(
    image_dir="data_v3_processed/val/images/images",
    mask_dir="data_v3_processed/val/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
)

In [24]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=1,
                    batch_size=8,
                    steps_per_epoch=3000,
                    validation_steps=13)

3000/3000 [==============================] - 2871s 951ms/step - loss: 0.0418 - f1: 0.7742 - val_loss: 0.0437 - val_f1: 0.1226


In [25]:
model.save("lstm_unet_model.h5")